<a href="https://colab.research.google.com/github/ElhadjHoussem/Colab/blob/master/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_folder = "2"
model_name = "/FacialExpressionRecognition"
colab_save_path ="gdrive/MyDrive/GoogleColab/models/" 
! mkdir gdrive/MyDrive/GoogleColab/models/2

mkdir: cannot create directory ‘gdrive/MyDrive/GoogleColab/models/2’: File exists


In [ ]:
!pwd


/content


In [ ]:
! mkdir ~/.kaggle 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp /content/gdrive/MyDrive/GoogleColab/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              235KB  2021-09-09 20:40:01          12011        991  1.0              
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           7320        553  0.9375           
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2922        217  1.0              
dhruvildave/wikibooks-dataset                               Wikibooks Dataset             

In [ ]:
! kaggle datasets download -d ahmedmoorsy/facial-expression

facial-expression.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir downloads

mkdir: cannot create directory ‘downloads’: File exists


In [ ]:
! unzip facial-expression.zip -d downloads

Archive:  facial-expression.zip
replace downloads/fer2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace downloads/fer2013/fer2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n A


In [ ]:
import io
import pandas as pd

df = pd.read_csv("/content/downloads/fer2013.csv")

In [ ]:
%tensorflow_version 1.10

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.10`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
pip install h5py==2.10.0

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.


In [ ]:
!pip install keras2onnx

     |████████████████████████████████| 96 kB 3.2 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 12.3 MB 175 kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=ae028153f0d95b1638e6baa60c837b5c02358ed1bf7cb8a3edd097e41d39f588
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
keras.__version__
print(tf.__version__,keras.__version__)

1.15.2 2.2.4-tf


In [ ]:
import sys, os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.models import model_from_json


In [ ]:
print("start")
print(df.head())


start
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [ ]:
row = df["pixels"][0]

pixels = np.array(row.split(" "),'float32')
pixels.shape


(2304,)

In [ ]:
48*48

2304

In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [ ]:
num_labels = 7
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


#normalizing data between o and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], width, height, 1)

X_test = X_test.reshape(X_test.shape[0], width, height, 1)

print(f"shape:{X_train.shape}")
print(f"shape:{train_y.shape}")


shape:(28709, 48, 48, 1)
shape:(28709, 7)


In [ ]:


##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(32,kernel_size= (5, 5), activation='relu'))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

#Compliling the model
model.compile(loss=categorical_crossentropy,
            optimizer=Adam(),
            metrics=['accuracy'])

model.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 44, 44, 32)        832       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 40, 40, 32)        25632     
_________________________________________________________________
batch_normalization_7 (Batch (None, 40, 40, 32)        128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 38, 38, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 36, 36, 64)        36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 36, 36, 64)       

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13826131673188989058
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14215605667650958648
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11643268194507421227
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11338832282
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9067640422104346159
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [ ]:
epochs = 200
batch_size = 64

with(tf.device('GPU:0')):
    #Training the model
  model.fit(X_train, train_y,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, test_y), shuffle=True)


Train on 28709 samples, validate on 3589 samples
Epoch 1/200
28709/28709 [==============================] - 35s 1ms/step - loss: 1.9313 - accuracy: 0.3003 - val_loss: 1.6439 - val_accuracy: 0.3547
Epoch 2/200
28709/28709 [==============================] - 34s 1ms/step - loss: 1.5324 - accuracy: 0.4008 - val_loss: 1.5206 - val_accuracy: 0.4118
Epoch 3/200
28709/28709 [==============================] - 34s 1ms/step - loss: 1.4059 - accuracy: 0.4509 - val_loss: 1.3987 - val_accuracy: 0.4547
Epoch 4/200
28709/28709 [==============================] - 34s 1ms/step - loss: 1.3023 - accuracy: 0.4985 - val_loss: 1.3108 - val_accuracy: 0.5007
Epoch 5/200
28709/28709 [==============================] - 34s 1ms/step - loss: 1.2076 - accuracy: 0.5367 - val_loss: 1.3019 - val_accuracy: 0.5015
Epoch 6/200
28709/28709 [==============================] - 34s 1ms/step - loss: 1.1144 - accuracy: 0.5727 - val_loss: 1.2497 - val_accuracy: 0.5272
Epoch 7/200
28709/28709 [==============================] - 34s 

In [ ]:
model_save_path = colab_save_path + model_folder + model_name 


In [ ]:
# serialize model to JSON

model_json = model.to_json()
with open(model_save_path+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_save_path+".h5")
print("Saved model to disk in =>" + model_save_path)

Saved model to disk in =>gdrive/MyDrive/GoogleColab/models/2/FacialExpressionRecognition


**Convert keras Model to Onnx**

In [ ]:
model_save_path

'gdrive/MyDrive/GoogleColab/models/2/FacialExpressionRecognition'

In [ ]:
import onnx
import os
import keras2onnx
from keras.models import model_from_json

os.environ['TF_KERAS'] = '1'

onnx_model_name = model_name+'.onnx'
model = model_from_json(open(model_save_path+".json", "r").read())

#load weights
h5_model=model_save_path+".h5"
model.load_weights(h5_model)
onnx_model = keras2onnx.convert_keras(model, model.name,target_opset=9)
onnx.save_model(onnx_model, model_save_path+'.onnx')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



The ONNX operator number change on the optimization: 59 -> 29
